In [46]:
import graph_tool.all as gt
from hSBM_Topicmodel.sbmtm import sbmtm

import pandas as pd
from collections import Counter
import nltk
import string
import regex as re
import numpy as np
import ast

from spacy.lang.da.stop_words import STOP_WORDS

## Preprocessing

In [17]:
df = pd.read_csv("tweets_final.zip", compression='zip')
df.head()

/Users/jeppefoldberg/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_full_text,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,RT @madsnyvold: Nogle sukker over stormflod og...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold']
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,@COWIdk konkluderer at @kystdk's højvandsstati...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,Forskellen er at @Cowidk inkluderer historiske...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,@kristianpagh Kender du denne rapport for KBH?...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,@kristianpagh er den for høj i køge? -Kbh er j...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']


In [19]:
# removing duplicates from 
df.drop_duplicates(subset='tweet_id') 

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_full_text,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,RT @madsnyvold: Nogle sukker over stormflod og...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold']
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,@COWIdk konkluderer at @kystdk's højvandsstati...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,Forskellen er at @Cowidk inkluderer historiske...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,@kristianpagh Kender du denne rapport for KBH?...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,@kristianpagh er den for høj i køge? -Kbh er j...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340366,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Sun Mar 14 11:55:43 +0000 2021,...,Vi lancerer i dag stort udspil for plantebaser...,False,{},16.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340367,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Mon Mar 15 11:26:58 +0000 2021,...,Aftale om sommerens festivaler præsenteres om ...,False,{},7.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340368,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Tue Apr 27 20:05:33 +0000 2021,...,"Tror I, at vi er dumme? Regeringen reklamerer ...",False,{},33.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340369,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Wed Apr 28 06:01:04 +0000 2021,...,"@hansersej Det er vi fuldt bevidste om, hvilke...",False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['hansersej']


In [20]:
def preprocess(text, tokenizer=nltk.tokenize.casual.TweetTokenizer()):
    '''takes in text and removes urls, mentions, numbers and punctuation'''
    if type(text) == str:
        text = text.lower()
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text)
        text = re.sub(r'https?', '', text)  # removes the extra weird https left in the text. 
        text = re.sub(r'\.\.\.', '', text)  # we do not care if people use ...
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[0-9]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = tokenizer.tokenize(text)
    else: 
        return np.nan
    return text

In [21]:
df['preprocessed_text'] = df.tweet_full_text.apply(preprocess)

In [18]:
df.to_csv('all_tweets_preprocessed.zip', compression='zip')

In [27]:
df

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions,preprocessed_text
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold'],"[RT, Nogle, sukker, over, stormflod, og, høj, ..."
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']","[konkluderer, at, s, højvandsstatistik, undere..."
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']","[Forskellen, er, at, inkluderer, historiske, k..."
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh'],"[Kender, du, denne, rapport, for, KBH]"
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh'],"[er, den, for, høj, i, køge, Kbh, er, jo, dere..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340366,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Sun Mar 14 11:55:43 +0000 2021,...,False,{},16.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Vi, lancerer, i, dag, stort, udspil, for, pla..."
340367,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Mon Mar 15 11:26:58 +0000 2021,...,False,{},7.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Aftale, om, sommerens, festivaler, præsentere..."
340368,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Tue Apr 27 20:05:33 +0000 2021,...,False,{},33.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Tror, I, at, vi, er, dumme, Regeringen, rekla..."
340369,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Wed Apr 28 06:01:04 +0000 2021,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['hansersej'],"[Det, er, vi, fuldt, bevidste, om, hvilket, ku..."


In [28]:
tokens = df['preprocessed_text']
handles = df['user_name']

In [29]:
hsbm_df = pd.DataFrame({'handles': handles, 'tokens': tokens})

In [33]:
hsbm_df_new = pd.read_csv('hsbm_df.zip', compression='zip', index_col=0)

In [45]:
hsbm_df_new.loc[hsbm_df_new.tokens.str.contains('http')]

,handles,tokens
31,Aslak Grinsted,"['RT', 'colleague', 'Jørgen', 'Peder', 'Steffe..."
39,Aslak Grinsted,"['RT', 'København', 'bliver', 'formentlig', 'l..."
73,Aslak Grinsted,"['RT', 'Läntagletscher', 'glacier', 'Switzerla..."
168,Anders Stouge,"['RT', 'TVavisen', 'lagde', 'vejen', 'forbi', ..."
171,Anders Stouge,"['RT', 'HKH', 'Kronprinsen', 'uddeler', 'pris'..."
...,...,...
339856,Zenia Stampe,"['RT', 'Dalai', 'Lama', 'har', 'en', 'forsonen..."
339877,Zenia Stampe,"['RT', 'Dejlig', 'tur', 'i', 'Mjølnerparken', ..."
339938,Zenia Stampe,"['RT', 'Zenia', 'Stampe', 'om', 'krisen', 'hos..."
340020,Zenia Stampe,"['RT', 'Støjberg', 'mener', 'at', 'forholdene'..."


In [26]:
# save it to 
compression = dict(method='zip', archive_name= 'hsbm_df.csv')
hsbm_df.to_csv('hsbm_df.zip', compression=compression)

## Getting the data in a format usable for the hSBM

In [48]:
def extract_dict(string):
    """Helper function to extract dict from string if string exists 
    else the function returns an empty dict (Thanks, Tweepy...)
    Here it is used to extract lists; that works?!"""
    try:
        out = ast.literal_eval(string)
    except:
        out = dict()
    return out

#hsbm_df["tokens"] = hsbm_df.tokens.apply(lambda x: extract_dict(x))

In [ ]:
handles = df['handles']
docs = df['tokens']
# stopwords imported in the beginning; we wan't to remove them! 
docs = [[word for word in doc if word not in STOP_WORDS] for doc in docs]

In [ ]:
# Remove infrequent words; Snorres solution
cutoff = 5
c = Counter()
for doc in docs:
    c.update(Counter(doc))
vocab = c.most_common(25000)
vocab = set([word for word, count in vocab if count > 5])
# remove words
docs = [[w for w in doc if w in vocab] for doc in docs]

# Training the hSBM model

In [ ]:
# step 3
model = sbmtm()

## we have to create the document network from the corpus
model.make_graph(docs)

gt.seed_rng(42) ## seed for graph-tool's random number generator --> same results
model.fit()

In [ ]:
model.save_graph(filename = 'graph_full_dataset_nostopwords.xml.gz')
model.topics

In [ ]:
def get_topic_df(topics):
  '''takes in model.topic and returns a df that can be used to save the topics 
  could (and probably should) be rewritten with list comprehensions!'''
  topic_nr = []
  words = []
  weights = []
  for topic in topics: 
    for word in topics[topic]:
      topic_nr.append('topic ' + str(topic))
      words.append(word[0])
      weights.append(word[1])

  return pd.DataFrame({'topic_nr': topic_nr,
                'words': words,
                'weights': weights})

In [ ]:
topic_df = get_topic_df(topics)

In [ ]:
topic_df.to_csv('topics_6000_nostopwords.csv')

## Finding the green words in the topics from hSBM

In [5]:
# reading in the topics from the largest model! 
all_topics = pd.read_csv('topics_100_000_nostopwords.csv', index_col=0)

In [12]:
# We have 94 topics
all_topics.tail()

,topic_nr,words,weights
916,topic 94,mundbind,0.026020
917,topic 94,Eksperter,0.024735
918,topic 94,Analyse,0.023771
919,topic 94,banker,0.023129
920,topic 94,Professor,0.018953


In [40]:
def add_words(topic_df):
    temp_words = []
    for word in topic_df.words:
        print(word, '\n')
        inp = input('\n\nIs this a green word: y or n?\n')
        if inp == 'y':
            temp_words.append(word)
        elif inp == 'n':
            continue
        else:
            print('I did not understand that!\n')
    return temp_words

In [41]:
def check_topics(n_topics, df):
    all_words = []
    for i in range(n_topics):
        # iterates over each topic nr and returns a df with that topic
        topic_df = all_topics.loc[all_topics.topic_nr == f'topic {i+1}']
        print(topic_df)
        inp = input('\n\nDoes this topic include a green word? y or n?\n')
        if inp == 'y':
            all_words += add_words(topic_df)
        elif inp == 'n':
            continue
        else:
            print('I did not understand that!')
    return all_words


all_words = check_topics(94, all_topics)

   topic_nr    words   weights
10  topic 1      Men  0.051774
11  topic 1       Og  0.044839
12  topic 1     helt  0.027309
13  topic 1    mener  0.015898
14  topic 1  handler  0.008996
15  topic 1  Hvorfor  0.008480
16  topic 1     Hvor  0.008191
17  topic 1     Ikke  0.008180
18  topic 1      set  0.007761
19  topic 1      Man  0.007160


Does this topic include a green word? y or n?
n
   topic_nr          words   weights
20  topic 2             DK  0.452111
21  topic 2          grønt  0.128635
22  topic 2       grønnere  0.055364
23  topic 2            øge  0.050271
24  topic 2           Brug  0.033185
25  topic 2       bidrager  0.031378
26  topic 2  foregangsland  0.030721
27  topic 2        globalt  0.030393
28  topic 2             fl  0.026121
29  topic 2           øger  0.025628


Does this topic include a green word? y or n?
y
DK 



Is this a green word: y or n?
n
grønt 



Is this a green word: y or n?
y
grønnere 



Is this a green word: y or n?
y
øge 



Is this a green wo



Does this topic include a green word? y or n?
n
     topic_nr       words   weights
178  topic 19         Jeg  0.118452
179  topic 19          Du  0.024534
180  topic 19        tror  0.020920
181  topic 19        folk  0.018557
182  topic 19        vist  0.012811
183  topic 19        læse  0.011491
184  topic 19         dit  0.010031
185  topic 19  simpelthen  0.009267
186  topic 19     skrevet  0.009128
187  topic 19    egentlig  0.007854


Does this topic include a green word? y or n?
n
     topic_nr          words   weights
188  topic 20           fmdk  0.178708
189  topic 20          debat  0.163421
190  topic 20            Kom  0.095905
191  topic 20          forbi  0.048954
192  topic 20          Debat  0.036943
193  topic 20            Mød  0.020200
194  topic 20            hør  0.019654
195  topic 20        program  0.018198
196  topic 20  spisekammeret  0.015469
197  topic 20     Folkemødet  0.013467


Does this topic include a green word? y or n?
n
     topic_nr       words



Does this topic include a green word? y or n?
n
     topic_nr words   weights
328  topic 34     🇩  0.321578
329  topic 34     🇰  0.309300
330  topic 34     🇪  0.082811
331  topic 34     🇺  0.079154
332  topic 34     🇸  0.038140
333  topic 34     🇳  0.019854
334  topic 34     🇬  0.014890
335  topic 34     🇧  0.012800
336  topic 34     🇮  0.011755
337  topic 34     🇷  0.011755


Does this topic include a green word? y or n?
n
     topic_nr     words   weights
338  topic 35      mand  0.049091
339  topic 35  personer  0.029894
340  topic 35       død  0.029308
341  topic 35      årig  0.026524
342  topic 35     årige  0.023154
343  topic 35      døde  0.021249
344  topic 35    kvinde  0.019637
345  topic 35      barn  0.017585
346  topic 35     dræbt  0.017292
347  topic 35   familie  0.016266


Does this topic include a green word? y or n?
n
     topic_nr words   weights
348  topic 36   the  0.070641
349  topic 36    in  0.053055
350  topic 36    of  0.048882
351  topic 36    on  0.043



Does this topic include a green word? y or n?
n
     topic_nr           words   weights
480  topic 50           dkbiz  0.257802
481  topic 50           skabe  0.074284
482  topic 50           vækst  0.073749
483  topic 50       forskning  0.046815
484  topic 50             job  0.044998
485  topic 50  arbejdspladser  0.041578
486  topic 50         eksport  0.027576
487  topic 50            jobs  0.023621
488  topic 50           dkøko  0.023301
489  topic 50         styrker  0.017315


Does this topic include a green word? y or n?
n
     topic_nr         words   weights
490  topic 51         Dansk  0.170663
491  topic 51    Folkeparti  0.040816
492  topic 51        Arktis  0.026531
493  topic 51  Konservative  0.025255
494  topic 51       Erhverv  0.016837
495  topic 51           HKH  0.016327
496  topic 51      Frederik  0.012500
497  topic 51           vor  0.012500
498  topic 51          tids  0.011735
499  topic 51         Hvert  0.010969


Does this topic include a green word? y 



Does this topic include a green word? y or n?
n
     topic_nr     words   weights
630  topic 65     søger  0.058207
631  topic 65     video  0.038621
632  topic 65    Martin  0.034207
633  topic 65   kollega  0.031172
634  topic 65         😁  0.029241
635  topic 65      Mads  0.024552
636  topic 65  stilling  0.024000
637  topic 65   Nielsen  0.024000
638  topic 65      COWI  0.016276
639  topic 65         🤣  0.015172


Does this topic include a green word? y or n?
n
     topic_nr          words   weights
640  topic 66          Mette  0.188101
641  topic 66    Frederiksen  0.117701
642  topic 66          Løkke  0.082225
643  topic 66           Lars  0.076312
644  topic 66              F  0.049704
645  topic 66  statsminister  0.046009
646  topic 66         Jensen  0.042313
647  topic 66         Morten  0.035107
648  topic 66       Kristian  0.023836
649  topic 66          Frank  0.018662


Does this topic include a green word? y or n?
n
     topic_nr           words   weights
650  to

     topic_nr            words   weights
760  topic 78      Folketinget  0.223148
761  topic 78         klimalov  0.171529
762  topic 78         vedtaget  0.058673
763  topic 78    borgerforslag  0.052902
764  topic 78       klimaloven  0.045527
765  topic 78  borgerforslaget  0.040077
766  topic 78      folketinget  0.036550
767  topic 78         bindende  0.032382
768  topic 78    underskrifter  0.030458
769  topic 78             Støt  0.023405


Does this topic include a green word? y or n?
y
Folketinget 



Is this a green word: y or n?
n
klimalov 



Is this a green word: y or n?
y
vedtaget 



Is this a green word: y or n?
n
borgerforslag 



Is this a green word: y or n?
n
klimaloven 



Is this a green word: y or n?
y
borgerforslaget 



Is this a green word: y or n?
n
folketinget 



Is this a green word: y or n?
n
bindende 



Is this a green word: y or n?
n
underskrifter 



Is this a green word: y or n?
n
Støt 



Is this a green word: y or n?
n
     topic_nr         words 

økonomi 



Is this a green word: y or n?
n
affald 



Is this a green word: y or n?
n
plast 



Is this a green word: y or n?
n
genanvendelse 



Is this a green word: y or n?
y
havet 



Is this a green word: y or n?
n
cirkulærøkonomi 



Is this a green word: y or n?
y
plastik 



Is this a green word: y or n?
n
cirkulær 



Is this a green word: y or n?
n
♻ 



Is this a green word: y or n?
n
kommunale 



Is this a green word: y or n?
n
     topic_nr words   weights
881  topic 91     ‍  0.268732
882  topic 91     ♂  0.099424
883  topic 91     🏽  0.077810
884  topic 91     ♀  0.072046
885  topic 91     🌾  0.049712
886  topic 91     🤷  0.048271
887  topic 91     🤦  0.047550
888  topic 91     👩  0.046830
889  topic 91     👨  0.043948
890  topic 91     🐷  0.021614


Does this topic include a green word? y or n?
n
     topic_nr     words   weights
891  topic 92      hjem  0.254902
892  topic 92    Syrien  0.089076
893  topic 92     hente  0.071709
894  topic 92   syriske  0.035294
895 

In [50]:
temp = []
for word in all_words:
    temp.append(word.lower())
all_words_unique = set(temp)
all_words_unique

{'afgift',
 'biodiversitet',
 'biodiversitetskrise',
 'biogas',
 'biomasse',
 'bæredygtig',
 'bæredygtigt',
 'cirkulærøkonomi',
 'co',
 'coafgift',
 'coneutrale',
 'dkgreen',
 'dkklima',
 'dknatur',
 'dkvind',
 'elbil',
 'elbiler',
 'energi',
 'forbrug',
 'forurening',
 'fossil',
 'fossilbiler',
 'fossile',
 'genanvendelse',
 'grader',
 'grøn',
 'grønne',
 'grønnere',
 'grønomstilling',
 'grønt',
 'havvind',
 'huskklima',
 'klima',
 'klimaaftryk',
 'klimaansvar',
 'klimaet',
 'klimaforandringer',
 'klimaforandringerne',
 'klimakrisen',
 'klimalov',
 'klimaloven',
 'klimamål',
 'klimapolitik',
 'klimatal',
 'klimatopmøde',
 'klimatosse',
 'klimavenlig',
 'klimavenlige',
 'klimavenligt',
 'kloden',
 'kulkraftværker',
 'landbrugspakken',
 'madspild',
 'miljø',
 'miljøet',
 'natur',
 'naturen',
 'oliejagt',
 'oliejagten',
 'olieselskaber',
 'omstilling',
 'opvarmning',
 'parisaftalen',
 'reducere',
 'reduktion',
 'reduktioner',
 'udledning',
 'udledninger',
 'vedvarende',
 'vindmøller',
 '

In [52]:
with open("green_words.txt", "w") as output:
    output.write(str(all_words_unique))

## Finding additional keywords with W2V

In [120]:
with open('green_words.txt', 'r') as f:
    green_words = f.read()

green_words = extract_dict(green_words)

In [1]:
from gensim.models import KeyedVectors

/Users/jeppefoldberg/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
w2v = KeyedVectors.load_word2vec_format('w2v/dsl_skipgram_2020_m5_f500_epoch2_w5.model.w2v.bin', binary=True)

In [56]:
def expand_words(words, n):
    '''takes in a set of words and shows potential words to expand the set'''
    extra_words = []
    for word in words:
        try:
            similar_words = [word[0] for word in w2v.most_similar(positive=word, topn=n) if word[0] not in words]
        except:
            continue
        # also removes words that are already in extra words!
        similar_words = [word for word in similar_words if word not in extra_words]
        for i, word in enumerate(similar_words):
            print('\n', i, word, '\n')
        inp = input('Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:\n')
        indices = [int(i) for i in inp.split(' ') if i.isdigit()]
        for i in indices:
            extra_words.append(similar_words[i])
    return extra_words


In [57]:
extra_words = expand_words(green_words, 10)


 0 klimakrise 


 1 klimaproblemet 


 2 klimaudfordringen 


 3 klimaproblemerne 


 4 klimaproblemer 


 5 biodiversitetskrisen 


 6 klimaudfordringerne 


 7 klimakatastrofen 


 8 klimaudfordringer 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
1 2 3 4 5 6 7 8

 0 c3planterne 


 1 polnære 


 2 frekvenskomponenter 


 3 meridianbuer 


 4 yderteltet 


 5 nattilstand 


 6 antenneanalysator 


 7 crisprteknologi 


 8 hartkornsansættelser 


 9 erstatningskommission 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 opvarmningen 


 1 opvarming 


 2 drivhuseffekten 


 3 klimaændringer 


 4 drivhuseffekt 


 5 klimaændringerne 


 6 opvarmnings 


 7 temperaturstigninger 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
3 4 5 2 7

 0 gasjagt 


 1 olieefterforskningen 


 2 efterforskningsboringer 


 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
5

 0 klimalovgivning 


 1 klimatraktat 


 2 energilov 


 3 københavneraftale 


 4 immigrationsreform 


 5 energipakke 


 6 klimaplan 


 7 kyotoaftale 


 8 finansreform 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
0 1 3 6 7

 0 afgiften 


 1 afgifter 


 2 miljøafgift 


 3 energiafgift 


 4 ekstraafgift 


 5 elafgift 


 6 co2afgift 


 7 statsafgift 


 8 registreringsafgift 


 9 tillægsafgift 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
2 6

 0 klimamålsætninger 


 1 klimamålsætning 


 2 reduktionsmål 


 3 co2reduktionsmål 


 4 eumål 


 5 klimaplaner 


 6 miljømål 


 7 klimaforpligtelser 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
0 1 2 3 5 7

 0 energiafgift 


 1 co2afgiften 


 0 hybridbil 


 1 ampera 


 2 brintbil 


 3 prius 


 4 hybridbilen 


 5 pluginhybrid 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 ørkenklima 


 1 kontinentalklima 


 2 kystklima 


 3 klimaforhold 


 4 klimaklassifikation 


 5 köppens 


 6 subarktisk 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 kildesortering 


 1 genanvende 


 2 genanvendelsen 


 3 genanvendes 


 4 genanvendeligt 


 5 husholdningsaffaldet 


 6 genbrug 


 7 husholdningsaffald 


 8 recirkulering 


 9 pvcaffald 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
1 2 3 4 6 8

 0 olieefterforskning 


 1 gasefterforskning 


 2 olierekord 


 3 gaseventyr 


 4 rederigigant 


 5 gasprojekter 


 6 gasefterforskningen 


 7 gasplatforme 

Are any of these words related to the green debate? Input index numbers of the w

In [59]:
len(extra_words)

160

In [64]:
all_words_combined = extra_words + list(green_words)

In [68]:
all_words_combined = set(all_words_combined)

## Adding keywords from hSBM and W2V to the keywords from qualitative methods 

In [102]:
qual_words = pd.read_excel('keywords_qual.xlsx', header=None)

In [103]:
qual_words

,0
0,ClimateAction
1,ClimateEmergency
2,dkenergi
3,dkgreen
4,dkklima
...,...
150,vindmøllevinger
151,🌍
152,CCU-teknologier
153,🇩🇰


In [104]:
qual_words_set = set(qual_words[0].values)

In [105]:
print(len(qual_words_set))
print(len(qual_words_set - all_words_combined))

151
109


In [106]:
temp = qual_words_set - all_words_combined

In [107]:
final_word_list = set(list(temp) + list(all_words_combined))

In [108]:
len(final_word_list)

346

In [109]:
final_word_list

{0.7,
 '1,5-graders',
 2030,
 '2030-målet',
 '2udslip',
 '2udslippet',
 '70-procent',
 '70-procentsmål',
 'Bæredygtig',
 'CCS',
 'CCU-teknologier',
 'CO2',
 'CO2e-udledning',
 'ClimateAction',
 'ClimateEmergency',
 'Co2-reduktion',
 'Co2e',
 'DanJoergensen',
 'Fødevarer',
 'Hockeystav',
 'Klimaeffektiv',
 'Klimaeffektivt',
 'Klimaforandring',
 'Klimaforandringer',
 'Klimahandling',
 'Klimahandlingsplan',
 'Klimaindsatsen',
 'Klimamin',
 'Klimaminister',
 'Klimaministeriet',
 'Landbrug',
 'Miljø',
 'Miljøstyrelsen',
 'Natur',
 'Skov',
 'Udledning',
 'Vedvarende',
 'afgift',
 'benzinbiler',
 'biobrændsel',
 'biobrændsler',
 'biodieslen',
 'biodiversitet',
 'biodiversitetskrise',
 'biodiversitetskrisen',
 'biogas',
 'biogassen',
 'biomasse',
 'biomassen',
 'brintbiler',
 'bæredygtig',
 'bæredygtige',
 'bæredygtighed',
 'bæredygtigt',
 'cirkulærøkonomi',
 'co',
 'co2',
 'co2 kvote',
 'co2 kvoter',
 'co2-kvote',
 'co2-kvoter',
 'co2afgift',
 'co2afgiften',
 'co2afgifter',
 'co2aftryk',
 'co

In [116]:
df = pd.DataFrame(final_word_list)

In [119]:
df.to_csv('final_keywords.csv')